# Main Driver Code for Fernet

In [1]:
import base64
import binascii
import hmac
import time
import os
import struct
from pyaes import AESModeOfOperationCBC, Encrypter, Decrypter
import hashlib
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.backends import default_backend

__all__ = [
    "InvalidSignature",
    "InvalidToken",
    "Fernet"
]
_MAX_CLOCK_SKEW = 60

class InvalidToken(Exception):
    pass

class InvalidSignature(Exception):
    pass

class Fernet:
    def __init__(self, key):
        if not isinstance(key, bytes):
            raise TypeError("key must be bytes.")

        key = base64.urlsafe_b64decode(key)
        if len(key) != 32:
            raise ValueError("Fernet key must be 32 url-safe base64-encoded bytes.")

        self._signing_key = key[:16]
        self._encryption_key = key[16:]

    @classmethod
    def generate_key(cls):
        return base64.urlsafe_b64encode(os.urandom(32))

    def encrypt(self, data):
        current_time = int(time.time())
        iv = os.urandom(16)
        return self._encrypt_from_parts(data, current_time, iv)

    def _encrypt_from_parts(self, data, current_time, iv):
        encrypter = Encrypter(AESModeOfOperationCBC(self._encryption_key, iv))
        ciphertext = encrypter.feed(data)
        ciphertext += encrypter.feed()

        basic_parts = (b"\x80" + struct.pack(">Q", current_time) + iv + ciphertext)

        hmactext = hmac.new(self._signing_key, digestmod='sha256')
        hmactext.update(basic_parts)

        return base64.urlsafe_b64encode(basic_parts + hmactext.digest())

    def decrypt(self, token, ttl=None):
        if not isinstance(token, bytes):
            raise TypeError("token must be bytes.")

        current_time = int(time.time())

        try:
            data = base64.urlsafe_b64decode(token)
        except (TypeError, binascii.Error):
            raise InvalidToken

        if not data or data[0] != 0x80:
            raise InvalidToken

        try:
            timestamp, = struct.unpack(">Q", data[1:9])
        except struct.error:
            raise InvalidToken
        if ttl is not None:
            if timestamp + ttl < current_time:
                raise InvalidToken

            if current_time + _MAX_CLOCK_SKEW < timestamp:
                raise InvalidToken

        hmactext = hmac.new(self._signing_key, digestmod='sha256')
        hmactext.update(data[:-32])
        if not hmac.compare_digest(hmactext.digest(), data[-32:]):
            raise InvalidToken

        iv = data[9:25]
        ciphertext = data[25:-32]
        decryptor = Decrypter(AESModeOfOperationCBC(self._encryption_key, iv))
        try:
            plaintext = decryptor.feed(ciphertext)
            plaintext += decryptor.feed()
        except ValueError:
            raise InvalidToken

        return plaintext

All outputs are in bytes so Python3 prints the string with a "b" in front, to remove that, we write the function RemoveB

In [2]:
def RemoveB(s):
    return(str(s).split("'")[1])

# Code to encrypt user given input using Fernet ( also decrypts at the end )

In [3]:
def get_key(password):
    digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
    digest.update(password)
    return base64.urlsafe_b64encode(digest.finalize())

val = input("TEXT:")
password = input("\n PASSWORD:")

if (len(password)>1):
	key = get_key(password.encode())
else:
	key = Fernet.generate_key()


print("\nKey: ",RemoveB(binascii.hexlify(bytearray(key))))


cipher_suite = Fernet(key)
cipher_text = cipher_suite.encrypt(val.encode())
cipher=binascii.hexlify(bytearray(cipher_text))
print("\nCipher: ",RemoveB(cipher))

print("\nVersion:\t",RemoveB(cipher[0:2]))
print("\nTime stamp:\t",RemoveB(cipher[2:18]))
print("\nIV:\t\t",RemoveB(cipher[18:50]))
print("\nHMAC:\t\t",RemoveB(cipher[-64:]))

plain_text = cipher_suite.decrypt(cipher_text)
print("\n =================== \n Decrypted text: ",RemoveB(plain_text))

TEXT:May The force be with you

 PASSWORD:Yoda

Key:  32544364424c754d6c4767564d30304f6955725f66745667466754636d73447078376f654b6865434d63673d

Cipher:  674141414141426567616b72724a75475f474c3941587061397a68674f72323663474164495333586a377a70614967584f5062564f4275625137346e736b7a776b7863505452724c343939796d30436b6b30645a5f587a567975613749616d36337243565637736e5934376e4152435966416c326b62633d

Version:	 67

Time stamp:	 4141414141426567

IV:		 616b72724a75475f474c394158706139

HMAC:		 7975613749616d36337243565637736e5934376e4152435966416c326b62633d

 Decrypted text:  May The force be with you
